In [159]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tweepy
import re
import emoji
import nltk
nltk.download('words')
words = set(nltk.corpus.words.words())
stopword = nltk.corpus.stopwords.words('english')

import glob
import string

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

[nltk_data] Downloading package words to C:\Users\SweetY
[nltk_data]     pAtel\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
movieInfoFile = 'data_files/u.item'
movieInfo = pd.read_csv(movieInfoFile, sep="|", header=None, index_col=False,
                  names=['itemId','title'], usecols=[0,1])
movieInfo

,itemId,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)
...,...,...
1677,1678,Mat' i syn (1997)
1678,1679,B. Monkey (1998)
1679,1680,Sliding Doors (1998)
1680,1681,You So Crazy (1994)


In [3]:
movieInfo['year'] = movieInfo.title.str.extract('(\(\d\d\d\d\))',expand=False)
movieInfo['year'] = movieInfo.year.str.extract('(\d\d\d\d)',expand=False)

#Removing the years from the 'title' column
movieInfo['title'] = movieInfo.title.str.replace('(\(\d\d\d\d\))', '')

<ipython-input-3-2d0fe1d6a320>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  movieInfo['title'] = movieInfo.title.str.replace('(\(\d\d\d\d\))', '')


In [4]:
movieInfo

,itemId,title,year
0,1,Toy Story,1995
1,2,GoldenEye,1995
2,3,Four Rooms,1995
3,4,Get Shorty,1995
4,5,Copycat,1995
...,...,...,...
1677,1678,Mat' i syn,1997
1678,1679,B. Monkey,1998
1679,1680,Sliding Doors,1998
1680,1681,You So Crazy,1994


In [20]:
movieInfo['tagword'] = movieInfo['title'].str.lower()
movieInfo['tagword'] = movieInfo.tagword.str.replace(' ', '')
movieInfo['tagword'] = movieInfo['tagword'].apply(lambda x: "{}{}".format('#', x))
movieInfo['tagword'] = movieInfo.tagword.str.replace('(\([A-Za-z0-9_]+\))', '')
movieInfo

<ipython-input-20-657f3eff8acf>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  movieInfo['tagword'] = movieInfo.tagword.str.replace('(\([A-Za-z0-9_]+\))', '')


,itemId,title,year,tagword
0,1,Toy Story,1995,#toystory
1,2,GoldenEye,1995,#goldeneye
2,3,Four Rooms,1995,#fourrooms
3,4,Get Shorty,1995,#getshorty
4,5,Copycat,1995,#copycat
...,...,...,...,...
1677,1678,Mat' i syn,1997,#mat'isyn
1678,1679,B. Monkey,1998,#b.monkey
1679,1680,Sliding Doors,1998,#slidingdoors
1680,1681,You So Crazy,1994,#yousocrazy


In [181]:
movieInfo['title'][143]

'Sound of Music, The '

In [21]:
api_key = "0Nfp0QdlOHXy5FVWtI8T7xdct"
api_secret = "7IMXzjvfbStJejs1iI2B75esFcGehySRvWil31ZKGl48yNfr9E"
access_token = "1354846100847550467-Lelq383ZqoVZDlRTyj9BfmVkWNyWyU"
access_secret = "vqAMs6mi3oQpkwboh554koYc1e3ScEXbBdyGRqRV2E1Q5"

In [22]:
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth)

In [48]:
# function to perform data extraction
def scrape(movieID, movie_title, words):
    numtweet = 100  
    
    # Creating DataFrame using pandas
    db = pd.DataFrame(columns=['Movie Id', 'Title', 'Tagword', 'description', 'totaltweets', 'retweetcount', 'text', 'hashtags'])
      
    # We are using .Cursor() to search through twitter for the required tweets.
    # The number of tweets can be restricted using .items(number of tweets)
    tweets = tweepy.Cursor(api.search, q=words, lang="en",
                           tweet_mode='extended').items(numtweet)
     
    # .Cursor() returns an iterable object. Each item in 
    # the iterator has various attributes that you can access to 
    # get information about each tweet
    list_tweets = [tweet for tweet in tweets]
      
    # Counter to maintain Tweet Count
    i = 1  
      
    # we will iterate over each tweet in the list for extracting information about each tweet
    for tweet in list_tweets:
        movie_id = movieID
        title = movie_title
        tagword = words
        description = tweet.user.description
        totaltweets = tweet.user.statuses_count
        retweetcount = tweet.retweet_count
        hashtags = tweet.entities['hashtags']
          
        # Retweets can be distinguished by a retweeted_status attribute,
        # in case it is an invalid reference, except block will be executed
        try:
            text = tweet.retweeted_status.full_text
        except AttributeError:
            text = tweet.full_text
        hashtext = list()
        for j in range(0, len(hashtags)):
            hashtext.append(hashtags[j]['text'])
          
        # Here we are appending all the extracted information in the DataFrame
        ith_tweet = [movie_id, title, tagword, description,
                     totaltweets, retweetcount, text, hashtext]
        db.loc[len(db)] = ith_tweet
          
        # Function call to print tweet data on screen
#         print(ith_tweet)
        i = i+1
    filename = str(movie_title) + '_tweets.csv'
      
    # we will save our database as a CSV file.
    db.to_csv('movie_tweets_collected/' + filename)

In [ ]:
for index, row in movieInfo[143:].iterrows():
    movieID = row['itemId']
    movie_title = row['title']
    movie_tagword = row['tagword']
    scrape(movieID, movie_title, movie_tagword)

# Working with the collected Tweets.....

In [69]:
# specifying the path to csv files
path = "movie_tweets_collected"
  
# csv files in the path
files = glob.glob(path + "/*.csv")
  
print(len(files))
  
# defining an empty list to store 
# content
data_frame = pd.DataFrame()
content = []
  
# checking all the csv files in the 
# specified path
for filename in files:
    
    # reading content of csv file
    # content.append(filename)
    df = pd.read_csv(filename, index_col=None)
    content.append(df)

    
# converting content to data frame
data_frame = pd.concat(content)

85


In [70]:
data_frame

,Unnamed: 0,Movie Id,Title,Tagword,description,totaltweets,retweetcount,text,hashtags
0,0,28,Apollo 13,#apollo13,I’m a theatre kid who loves space. Proud Democ...,5153,8,Final surprise for #Apollo13 celebrations. The...,['Apollo13']
1,1,28,Apollo 13,#apollo13,"Poet, Storyteller... at least trying to tell one.",1877,0,@noursafeialdin @FilmstoFilms_ One of my all t...,['Apollo13']
2,2,28,Apollo 13,#apollo13,An open source bot here to support all women w...,462150,1,"🎥 On #MovieMonday, we highlight movies that ha...",['MovieMonday']
3,3,28,Apollo 13,#apollo13,👩‍💻 Our mission is to inspire you to excel in ...,1906,1,"🎥 On #MovieMonday, we highlight movies that ha...","['MovieMonday', 'womenwhocode', 'Apollo13', 'J..."
4,4,28,Apollo 13,#apollo13,Log On http://t.co/Jd5OkgIQ3e And Stay Updated...,36243,0,@StevenDaGuy - Trill [ https://t.co/RLoE9yo3dq...,['Apollo13']
...,...,...,...,...,...,...,...,...,...
92,92,66,While You Were Sleeping,#whileyouweresleeping,Always contrary. No Dms please. I am Proficie...,398858,3,WAKE UP JORDY!! @J_Mei21\n\nYou gotta know wha...,"['Scheming', 'Sneaking', 'WhileYouWereSleeping']"
93,93,66,While You Were Sleeping,#whileyouweresleeping,Do not lead me down this road unless you’re su...,29899,3,WAKE UP JORDY!! @J_Mei21\n\nYou gotta know wha...,"['Scheming', 'Sneaking', 'WhileYouWereSleeping..."
94,94,66,While You Were Sleeping,#whileyouweresleeping,"Woof, Woof, Grrrrr. 🐾🐺🍖🍗\nI eat Noms | random ...",28346,0,@kpopxkdramas #whileyouweresleeping to me it i...,['whileyouweresleeping']
95,95,66,While You Were Sleeping,#whileyouweresleeping,simple in all aspect of life,1013,10,Funny Moment from While You Were Sleeping kdra...,"['kdrama', 'kdramas']"


In [151]:
def cleaner(tweet):
    # Remove @ sign
    tweet = re.sub("@[A-Za-z0-9]+","",tweet)
    
    # Remove http links
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet)
    
    # Join the sub strings generated from re.sub
#     tweet = " ".join(tweet.split())
    
    # Remove puncutations
    tweet  = "".join([char for char in tweet if char not in string.punctuation])
    tweet = re.sub('[0-9]+', '', tweet)
    
    # Remove Emojis
    tweet = ''.join(c for c in tweet if c not in emoji.UNICODE_EMOJI)
    regrex_pattern = re.compile(pattern = "["
                                u"\U0001F600-\U0001F64F"  # emoticons
                                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                u"\U00002702-\U000027B0"
                                u"\U000024C2-\U0001F251"
                                u"\U00002500-\U00002BEF"  # chinese char
                                u"\U00002702-\U000027B0"
                                u"\U00002702-\U000027B0"
                                u"\U000024C2-\U0001F251"
                                u"\U0001f926-\U0001f937"
                                u"\U00010000-\U0010ffff"
                                u"\u2640-\u2642"
                                u"\u2600-\u2B55"
                                u"\u200d"
                                u"\u23cf"
                                u"\u23e9"
                                u"\u231a"
                                u"\ufe0f"  # dingbats
                                u"\u3030"
                               "]+", flags = re.UNICODE)
    tweet = regrex_pattern.sub(r'',tweet)
    
    # Remove hashtag sign and keep the text
    tweet = tweet.replace("#", "").replace("_", " ")
    
    #tokenize
    tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet) \
         if w.lower() in words or not w.isalpha())
    
    return tweet

In [158]:
clean_tweets = []
for index, row in data_frame.iterrows():
    desc = data_frame['description'][index].values
    desc = " ".join(str(i) for i in desc)
#     print(desc)
    clean_desc = cleaner(desc)
    clean_tweets.append(clean_desc)
#     print(clean_desc)
    
    
data_frame['cleaned_tweets'] = clean_tweets
data_frame

,Unnamed: 0,Movie Id,Title,Tagword,description,totaltweets,retweetcount,text,hashtags,cleaned_tweets
0,0,28,Apollo 13,#apollo13,I’m a theatre kid who loves space. Proud Democ...,5153,8,Final surprise for #Apollo13 celebrations. The...,['Apollo13'],I ’ m a who space Democratic Artist Painter Ar...
1,1,28,Apollo 13,#apollo13,"Poet, Storyteller... at least trying to tell one.",1877,0,@noursafeialdin @FilmstoFilms_ One of my all t...,['Apollo13'],Poet Storyteller at least trying to tell one w...
2,2,28,Apollo 13,#apollo13,An open source bot here to support all women w...,462150,1,"🎥 On #MovieMonday, we highlight movies that ha...",['MovieMonday'],An open source bot here to support all who cod...
3,3,28,Apollo 13,#apollo13,👩‍💻 Our mission is to inspire you to excel in ...,1906,1,"🎥 On #MovieMonday, we highlight movies that ha...","['MovieMonday', 'womenwhocode', 'Apollo13', 'J...",Our mission is to inspire you to excel in tech...
4,4,28,Apollo 13,#apollo13,Log On http://t.co/Jd5OkgIQ3e And Stay Updated...,36243,0,@StevenDaGuy - Trill [ https://t.co/RLoE9yo3dq...,['Apollo13'],Log On And Stay With Steven Da Latest Merchand...
...,...,...,...,...,...,...,...,...,...,...
92,92,66,While You Were Sleeping,#whileyouweresleeping,Always contrary. No Dms please. I am Proficie...,398858,3,WAKE UP JORDY!! @J_Mei21\n\nYou gotta know wha...,"['Scheming', 'Sneaking', 'WhileYouWereSleeping']",Batman Forever Twitter Bot An community for of...
93,93,66,While You Were Sleeping,#whileyouweresleeping,Do not lead me down this road unless you’re su...,29899,3,WAKE UP JORDY!! @J_Mei21\n\nYou gotta know wha...,"['Scheming', 'Sneaking', 'WhileYouWereSleeping...",to excellence in the art form Everything about...
94,94,66,While You Were Sleeping,#whileyouweresleeping,"Woof, Woof, Grrrrr. 🐾🐺🍖🍗\nI eat Noms | random ...",28346,0,@kpopxkdramas #whileyouweresleeping to me it i...,['whileyouweresleeping'],All I do is tweet favorite and Oh and share my...
95,95,66,While You Were Sleeping,#whileyouweresleeping,simple in all aspect of life,1013,10,Funny Moment from While You Were Sleeping kdra...,"['kdrama', 'kdramas']",in Finance Author of paranormal suspense myste...


In [161]:
data_frame.to_csv('data_files/cleaned_movie_tweets')

# Sentiment Analysis

In [164]:
movie_tweets = pd.read_csv('data_files/cleaned_movie_tweets')
movie_tweets.drop(['Unnamed: 0', 'Unnamed: 0.1', 'totaltweets', 'retweetcount', 'text', 'hashtags'], axis='columns', inplace=True)
movie_tweets

,Movie Id,Title,Tagword,description,cleaned_tweets
0,28,Apollo 13,#apollo13,I’m a theatre kid who loves space. Proud Democ...,I ’ m a who space Democratic Artist Painter Ar...
1,28,Apollo 13,#apollo13,"Poet, Storyteller... at least trying to tell one.",Poet Storyteller at least trying to tell one w...
2,28,Apollo 13,#apollo13,An open source bot here to support all women w...,An open source bot here to support all who cod...
3,28,Apollo 13,#apollo13,👩‍💻 Our mission is to inspire you to excel in ...,Our mission is to inspire you to excel in tech...
4,28,Apollo 13,#apollo13,Log On http://t.co/Jd5OkgIQ3e And Stay Updated...,Log On And Stay With Steven Da Latest Merchand...
...,...,...,...,...,...
3955,66,While You Were Sleeping,#whileyouweresleeping,Always contrary. No Dms please. I am Proficie...,Batman Forever Twitter Bot An community for of...
3956,66,While You Were Sleeping,#whileyouweresleeping,Do not lead me down this road unless you’re su...,to excellence in the art form Everything about...
3957,66,While You Were Sleeping,#whileyouweresleeping,"Woof, Woof, Grrrrr. 🐾🐺🍖🍗\nI eat Noms | random ...",All I do is tweet favorite and Oh and share my...
3958,66,While You Were Sleeping,#whileyouweresleeping,simple in all aspect of life,in Finance Author of paranormal suspense myste...


In [165]:
# init the sentiment analyzer
sia = SentimentIntensityAnalyzer()
sia_score_pos = []
sia_score_neg = []
sia_score_neu = []
sia_label = []

In [166]:
for tweet in movie_tweets['cleaned_tweets']:
    score = sia.polarity_scores(tweet)
    sia_score_pos.append(score['pos'])
    sia_score_neg.append(score['neg'])
    sia_score_neu.append(score['neu'])
    if score['compound'] > 0:
        sia_label.append("Positive")
    elif score['compound'] < 0:
        sia_label.append("Negative")
    elif score['compound'] == 0:
        sia_label.append("Neutral")
    

print(len(sia_score_pos))
print(len(sia_score_neg))
print(len(sia_score_neu))
print(len(sia_label))

3960
3960
3960
3960


In [167]:
movie_tweets['Sentiment_score_pos'] = sia_score_pos
movie_tweets['Sentiment_score_neg'] = sia_score_neg
movie_tweets['Sentiment_score_neu'] = sia_score_neu
movie_tweets['Sentiment_label'] = sia_label

In [168]:
movie_tweets

,Movie Id,Title,Tagword,description,cleaned_tweets,Sentiment_score_pos,Sentiment_score_neg,Sentiment_score_neu,Sentiment_label
0,28,Apollo 13,#apollo13,I’m a theatre kid who loves space. Proud Democ...,I ’ m a who space Democratic Artist Painter Ar...,0.218,0.071,0.711,Positive
1,28,Apollo 13,#apollo13,"Poet, Storyteller... at least trying to tell one.",Poet Storyteller at least trying to tell one w...,0.228,0.099,0.672,Positive
2,28,Apollo 13,#apollo13,An open source bot here to support all women w...,An open source bot here to support all who cod...,0.205,0.064,0.731,Positive
3,28,Apollo 13,#apollo13,👩‍💻 Our mission is to inspire you to excel in ...,Our mission is to inspire you to excel in tech...,0.229,0.045,0.726,Positive
4,28,Apollo 13,#apollo13,Log On http://t.co/Jd5OkgIQ3e And Stay Updated...,Log On And Stay With Steven Da Latest Merchand...,0.200,0.025,0.775,Positive
...,...,...,...,...,...,...,...,...,...
3955,66,While You Were Sleeping,#whileyouweresleeping,Always contrary. No Dms please. I am Proficie...,Batman Forever Twitter Bot An community for of...,0.230,0.074,0.695,Positive
3956,66,While You Were Sleeping,#whileyouweresleeping,Do not lead me down this road unless you’re su...,to excellence in the art form Everything about...,0.163,0.063,0.773,Positive
3957,66,While You Were Sleeping,#whileyouweresleeping,"Woof, Woof, Grrrrr. 🐾🐺🍖🍗\nI eat Noms | random ...",All I do is tweet favorite and Oh and share my...,0.275,0.029,0.696,Positive
3958,66,While You Were Sleeping,#whileyouweresleeping,simple in all aspect of life,in Finance Author of paranormal suspense myste...,0.246,0.027,0.727,Positive


In [169]:
movie_tweets.to_csv('data_files/movie_tweets_with_SA.csv')